# Testing functionality of G Counter CRDT implementations

This code and document is made by Alexander Kahanek on 3/31/2021.

Resources used for this project are:

+ https://www.youtube.com/watch?v=iEFcmfmdh2w
+ https://www.youtube.com/watch?v=OOlnp2bZVRs 

## G-Counter implementation

The functionality of the G Counter is that it never decreases. Thus, it is an easy implementation to have multiple counters that keeps track of multiple states asynchronously and independently. This allows for chatter between two counters to periodically update and give fast estimates of the total count if the clients counter requests one. This gives the client a faster response time at the cost of not giving an exact value to the client, however it also creates great functionality for offline tallying as counters can merge nodes when they are back online.

In this scenario, each G Counter has only one unique node that it can update; however, in actuality the code base can support as many unique nodes as wanted. In practice a GCounter would be launched when a client opens, where the nodes would pertain to any always-increasing counters. This app would need to be opened online to initially connect; and would be getting chatter from other counters (i.e, merges) asynchronously. However, the benefits of using this CRDT implementation would allow the app to go offline and still provide updates to the client, as well as store their own updates to eventually be pushed to other counters.

The outline of the entire use case is:

+ set up G Counters
    - these counters can only update their unique nodes
    - however, they store the node values for all seen nodes

+ increment node values for individual G Counters, independently of others

+ asynchronously chatter between nodes to keep values updating in the background

+ merge G Counter nodes when chattering
    - takes the max value of each node
        + this works as G Counters in always-increasing, meaning the max value is always the most recent value that is seen

+ continue incrementing and merging counters

+ etc, etc,

## Testing

Here we are doing simple tests to make sure the basic implementation works correctly.

In [1]:
from g_counter import *
import random

G1 = GCounter(1)
G2 = GCounter(2)

print(f'checking to make sure the counter is initialized: G1 = {G1}')

# adding some nodes and incrementing to both
G1.add_node(1)
G2.add_node(2)

G1.increment(1)
G1.increment(1)
G1.increment(1)

G2.increment(2)

print('added nodes with the same ID to G1 and G2 and incremented them')
G1.print_gc()
G2.print_gc()

print('merging the two counters')
G1.merge(G2)
G1.print_gc()
G2.print_gc()



checking to make sure the counter is initialized: G1 = <g_counter.GCounter object at 0x0000022A69A99280>
added nodes with the same ID to G1 and G2 and incremented them
g counter node 1 = {1: 3}, sum = 3
g counter node 2 = {2: 1}, sum = 1
merging the two counters
g counter node 1 = {1: 3, 2: 1}, sum = 4
g counter node 2 = {1: 3, 2: 1}, sum = 4


Next, we will do some more complicated tests...

Lets simulate that we have 3 counters, which each keep track of one distinct counters.

+ GCounter 1 -> node 1
+ GCounter 2 -> node 2
+ GCounter 3 -> node 3

This means that only node 1 can be incremented by GCounter 1, and so on. From here we will run a random choice of 100 increments without any merges. We will then merge GCounter 1 and GCounter 2, add a new counter (GCounter 4 -> node 4) and randomly apply 1000 increments, then merge all 4 counters. We will not initialize any nodes to the counters that have not seen it before, meaning it will take in the new information on a merge and create that node counter on the fly.

This effectively simulates a real scenario usage of this CRDT implementation. It allows for new counters to be updated and added periodically. The goal for these tests is to end our final full update with all increments being accounted for. In total we will push 1100 increments.


In [2]:
# create counters
G1 = GCounter(1)
G2 = GCounter(2)
G3 = GCounter(3)

# initialize counters
G1.add_node(1)
G2.add_node(2)
G3.add_node(3)

counters = [G1, G2, G3]

for i in range(100):
    choice_update = random.randint(1,len(counters))
    counters[choice_update-1].increment(choice_update)

print('initialized and randomly incremented elements for three counters')
[G.print_gc() for G in counters]
print()

print('here we will merge counter 1 and 2')
G1.merge(G2)
[G.print_gc() for G in counters]
print()

print('next we added a fourth counter and incremented more nodes randomly')
G4 = GCounter(4)
G4.add_node(4)
counters.append(G4)

for i in range(1000):
    choice_update = random.randint(1,len(counters))
    counters[choice_update-1].increment(choice_update)

[G.print_gc() for G in counters]
print()

print('now lets merge counters 1 and 3, then counters 2 and 4')
G2.merge(G4)
G1.merge(G3)
[G.print_gc() for G in counters]
print()

print('now merge counters 1 and 4, then counters 2 and 3')
G1.merge(G4)
G2.merge(G3)
[G.print_gc() for G in counters]
print()

initialized and randomly incremented elements for three counters
g counter node 1 = {1: 35}, sum = 35
g counter node 2 = {2: 29}, sum = 29
g counter node 3 = {3: 36}, sum = 36

here we will merge counter 1 and 2
g counter node 1 = {1: 35, 2: 29}, sum = 64
g counter node 2 = {1: 35, 2: 29}, sum = 64
g counter node 3 = {3: 36}, sum = 36

next we added a fourth counter and incremented more nodes randomly
g counter node 1 = {1: 297, 2: 272}, sum = 569
g counter node 2 = {1: 297, 2: 272}, sum = 569
g counter node 3 = {3: 302}, sum = 302
g counter node 4 = {4: 229}, sum = 229

now lets merge counters 1 and 3, then counters 2 and 4
g counter node 1 = {1: 297, 2: 272, 4: 0, 3: 302}, sum = 871
g counter node 2 = {1: 297, 2: 272, 4: 229}, sum = 798
g counter node 3 = {1: 297, 2: 272, 4: 0, 3: 302}, sum = 871
g counter node 4 = {1: 297, 2: 272, 4: 229}, sum = 798

now merge counters 1 and 4, then counters 2 and 3
g counter node 1 = {1: 297, 2: 272, 4: 229, 3: 302}, sum = 1100
g counter node 2 = {

# Summary

As we see, each of our 4 counters correctly had a sum of 1100 at the end, meaning each counter fully accounted for every increment. As we kept information hidden and tried to replicate a real life implementation of the counters, this is great news!

One thing I do notice is that when updating the counters after the merge of counter 1 <-> 3, it added the node for counter 4 without interacting with any counters that had yet seen a node 4. I assume this is due to some python pointer issues; however, my base OOP knowledge on python is not deep enough to figure out why. Since this is a simple implementation, I will not attempt to fix that issue.